In [5]:
import requests
import numpy as np
import pandas as pd
import lxml.html as lxl
import re

In [6]:
MAX_PAGE = 1477

# 1. Functions

## Scraping the webpages to get all the car urls

In [7]:
## Get all the urls for all the listed used vehicles on truecar.com
def urls_scraping(base_url = 'https://bonbanh.com/oto-cu-da-qua-su-dung'):
    urls = []
    pages = []
    for i in range(1201, MAX_PAGE+1):
        pages.append(base_url + '/page,' + str(i))
    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
        except:
            break
        root = lxl.fromstring(response.content)
        url = ['https://bonbanh.com/' + link for link in root.xpath('//div[@id="s-list-car"]//a[@itemprop="url"]/@href')]
        urls += url

    return urls

## Parse one url to get information and return a dataframe

In [8]:
def convert_text_to_number(text):
  text = text.strip()
  # Kiểm tra xem chuỗi có khớp với định dạng số tiền không
  if("Tỷ" in text):
    match = re.match(r'^(\d+)(?: Tỷ)?(?: (\d{1,3}) Triệu)?$', text)
    if not match:
      print("So tien \'"+ text + "\' khong hop le")
      return 0
    else:
      billion_part, million_part = match.groups()
      # Lấy giá trị từ các nhóm trong kết quả khớp
      billion_part, million_part = match.groups()

      # Chuyển đổi tỷ thành đơn vị số và thêm vào triệu nếu có
      total_amount = int(billion_part) * 1000
      if million_part:
          total_amount += int(million_part)

      return total_amount
  else:
    match = re.match(r'^(\d+)(?: Triệu)?$', text)
    if not match:
      print("So tien \'"+ text + "\' khong hop le")
      return 0
    else:
      million_part = match.groups()
      # Lấy giá trị từ nhóm khớp
      million_part = match.group(1)

      # Chuyển đổi triệu thành đơn vị số
      total_amount = int(million_part)

      return total_amount



In [9]:
def page_scraping(urls):
  array = []
  index = 0
  n = len(urls)
  percent = n // 100 + 1
  for url in urls:
    index = index + 1
    try:
      response = requests.get(url)
      response.raise_for_status()
      root = lxl.fromstring(response.content)
      title = root.xpath('//div[@class="title"]/h1/text()')[0]
      make = root.xpath('//*[@id="wrapper"]/div[2]/span[3]/a/span/strong/text()')[0]
      model = root.xpath('//*[@id="wrapper"]/div[2]/span[4]/a/span/strong/text()')[0]
      name = title.replace("\n", "").replace("\t", "").split("-")[0]
      price_plain = title.replace("\n", "").replace("\t", "").split("-")[-1]
      price = convert_text_to_number(price_plain)
      year = root.xpath('//span[@class="inp"]/text()')[0].replace("\n", "").replace("\t", "").strip();
      engine = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[1]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      exterior_color = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[2]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      interior_color = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[3]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      mileage = int(root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[1]/div[3]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").split(" ")[0].replace(",", ""))
      num_seats = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[4]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip().split(" ")[0]
      num_doors = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[5]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip().split(" ")[0]
      origin = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[1]/div[4]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      style = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[1]/div[5]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      transmission = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[1]/div[6]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      drivetrain = root.xpath('/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[6]/div[2]/span/text()')[0].replace("\n", "").replace("\t", "").strip()
      if(index % percent == 0):
        print(str(int(index*100/n))+ "%")
      array.append({ 'name': name, 'make': make, 'model': model, 'price': price, 'year': year, 'engine': engine, 'exterior_color': exterior_color,
                    'interior_color': interior_color, 'mileage': mileage, 'num_seats': num_seats, 'num_doors': num_doors, 'origin': origin,
                    'style': style, 'transmission': transmission, 'drivetrain': drivetrain})
    except:
      print("An exception occurred with index: "+ str(index))
  return pd.DataFrame(array)
    # return root

## Use multi-processing to scrape all urls and merge into one dataframe

# 2. Main function

## Getting all urls

In [10]:
%%time
urls=urls_scraping() # extract all vehicle urls from allowed 1470 pages.
print(len(urls))

5553
CPU times: user 35.5 s, sys: 610 ms, total: 36.1 s
Wall time: 10min 4s


In [11]:
%%time
data = page_scraping(urls)

1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
An exception occurred with index: 814
An exception occurred with index: 815
15%
16%
17%
18%
19%
20%
21%
An exception occurred with index: 1185
An exception occurred with index: 1186
22%
23%
24%
25%
26%
An exception occurred with index: 1458
An exception occurred with index: 1459
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
An exception occurred with index: 2373
An exception occurred with index: 2374
43%
44%
45%
46%
47%
48%
49%
50%
51%
An exception occurred with index: 2858
An exception occurred with index: 2859
52%
53%
54%
An exception occurred with index: 3050
An exception occurred with index: 3051
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
An exception occurred with index: 3679
An exception occurred with index: 3680
66%
67%
68%
69%
70%
71%
72%
An exception occurred with index: 4060
An exception occurred with index: 4061
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
An exception occurred with index: 4628
An exception occurre

## Returning the raw dataframe

In [12]:
print(data.shape)
data.head(100)

(5527, 15)


,name,make,model,price,year,engine,exterior_color,interior_color,mileage,num_seats,num_doors,origin,style,transmission,drivetrain
0,Xe Mazda 6 2.0 AT 2016,Mazda,6,450,2016,Xăng 2.0 L,Trắng,Đen,100002,5,4,Lắp ráp trong nước,Sedan,Số tự động,FWD - Dẫn động cầu trước
1,Xe Peugeot 3008 AL 2022,Peugeot,3008,990,2022,Xăng 1.6 L,Trắng,Đen,6000,5,5,Lắp ráp trong nước,Crossover,Số tự động,FWD - Dẫn động cầu trước
2,Xe Ford Ranger Wildtrak 2.0L 4x4 AT 2018,Ford,Ranger,620,2018,Dầu 2.0 L,Cam,Đen,90000,5,4,Nhập khẩu,Bán tải / Pickup,Số tự động,4WD - Dẫn động 4 bánh
3,Xe Toyota Corolla altis 2.0V 2012,Toyota,Corolla altis,350,2012,Xăng 2.0 L,Vàng,Vàng,20,5,4,Lắp ráp trong nước,Sedan,Số tự động,FWD - Dẫn động cầu trước
4,Xe Mercedes Benz V class V250 Luxury 2021,Mercedes Benz,V class,2750,2021,Xăng 2.0 L,Đỏ,Nâu,28000,6,5,Nhập khẩu,Van/Minivan,Số tự động,RFD - Dẫn động cầu sau
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Xe Isuzu MU,Isuzu,MU-X,615,2021,Dầu 1.9 L,Trắng,-,0,7,5,Nhập khẩu,SUV,Số tay,RFD - Dẫn động cầu sau
96,Xe Toyota Innova 2.0E 2016,Toyota,Innova,339,2016,Xăng 2.0 L,Trắng,Kem,0,8,5,Lắp ráp trong nước,Crossover,Số tay,RFD - Dẫn động cầu sau
97,Xe Daewoo Matiz SE 0.8 MT 2005,Daewoo,Matiz,21,2005,Xăng 0.8 L,Xanh,-,0,5,5,Lắp ráp trong nước,Hatchback,Số tay,FWD - Dẫn động cầu trước
98,Xe Kia Spectra 1.6 MT 2004,Kia,Spectra,59,2004,Xăng 1.6 L,Bạc,Kem,0,5,4,Lắp ráp trong nước,Sedan,Số tay,FWD - Dẫn động cầu trước


In [13]:
data.to_csv('1201to1477.csv', encoding = 'utf-8')

In [ ]:
data.isna().sum()

name              0
make              0
model             0
price             0
year              0
engine            0
exterior_color    0
interior_color    0
mileage           0
num_seats         0
num_doors         0
origin            0
style             0
transmission      0
drivetrain        0
dtype: int64

In [ ]:
for column in data.columns:
  print(column)
  unique_values = data[column].unique()
  print(unique_values)

name
['Xe Ford Everest Titanium 2.0L 4x2 AT 2022' 'Xe Lexus RX 200t 2017'
 'Xe Mercedes Benz S class S450L Luxury 2020' ...
 'Xe Ford Focus 2.0 AT Ghia 2011' 'Xe Toyota Vios 1.5E 2011'
 'Xe Kia Cerato 1.6 AT Deluxe 2019']
make
['Ford' 'Lexus' 'Mercedes Benz' 'Toyota' 'Volvo' 'VinFast' 'LandRover'
 'BMW' 'Kia' 'Hyundai' 'Porsche' 'Mazda' 'Suzuki' 'Daewoo' 'Peugeot'
 'Mitsubishi' 'Audi' 'Nissan' 'Mini' 'Honda' 'Bentley' 'Chevrolet'
 'Subaru' 'Isuzu' 'Dongben' 'MG' 'Daihatsu' 'Thaco' 'Acura' 'Volkswagen'
 'Maserati' 'Dodge' 'Jeep' 'Rolls Royce' 'Cadillac' 'Infiniti'
 'Lamborghini' 'Renault' 'Citroen' 'Hummer' 'Lincoln' 'Jaguar' 'Zotye'
 'Ssangyong' 'Baic' 'SYM' 'Ferrari' 'Hino' 'Scion' 'Fiat' 'McLaren' 'RAM']
model
['Everest' 'RX' 'S class' 'Vios' 'S60' 'VF e34' 'Range Rover' 'Ranger'
 'X5' 'Sedona' 'SantaFe' 'GLC' '718' 'Cayenne' '2' 'Panamera' 'Kona'
 'Swift' 'Fadil' 'CX5' 'C class' 'Lacetti' 'Fortuner' 'Tucson' '3008'
 'Morning' 'Mustang' 'Pajero Sport' 'A6' 'V class' 'E class' '3' 'Ce